# Week 3: Peer Graded Assignment

### Creating a dataframe from html: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, 

In [1]:
!pip install beautifulsoup4
!pip install lxml

Requirement not upgraded as not directly required: beautifulsoup4 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: lxml in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [3]:
!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

Solving environment: done

# All requested packages already installed.



In [4]:
import csv

html_content = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(html_content, 'lxml')
#print(soup.prettify())
table = soup.find('table', class_ ='wikitable sortable')
table_rows = table.find_all('tr')

csv_file = open('Toronto_PostalCodes.csv','w')
csv_writer=csv.writer(csv_file)
csv_writer.writerow(['Postcode', 'Borough', 'Neighbourhood'])
for i, tr in enumerate (table_rows):
    if (i==0):
        th = tr.find_all('th')
        header =[i.text for i in th]
        #print(header)
       # csv_writer.writerow(header)
    else:
        td = tr.find_all('td')
        row =[i.text for i in td]
        #print(row)
        csv_writer.writerow(row)
        
csv_file.close()

In [5]:
# Reading the csv data
df=pd.read_csv('Toronto_PostalCodes.csv')

In [6]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


### Preprocessing of data

In [7]:
# stripping the '\n' character from the Neighbourhood column
df['Neighbourhood'] =df['Neighbourhood'].str.strip('\n')
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [8]:
# Replacing all Not assigned with numpy NaN

df.replace('Not assigned', np.nan, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
# dropping the rows which has Borough as NaN
new_df = df.dropna(subset=['Borough'], axis=0)
new_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [10]:
#Setting the neighbourhood as Borough's name whose Neighbourhood are NaN
clean_df = new_df.fillna(method='ffill', axis ='columns')
clean_df.head(15)

# Joining the Neighbourhoods with ',' of same borough
gp_df = clean_df[['Postcode','Borough','Neighbourhood']]
df2 = gp_df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Using the geocoder

In [11]:
!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

Solving environment: done

# All requested packages already installed.



In [12]:
# using the geocoder arcgis to get the latitude and longitude of all postal codes.

Postcode_latitude=[]
Postcode_longitude =[]

for i,postcode in enumerate(df2['Postcode']):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(df2['Postcode'][i]))
    lat_lng_coords = g.latlng
    Postcode_latitude.append(g.latlng[0])
    Postcode_longitude.append(g.latlng[1])
    
    
df2['Latitude'] = Postcode_latitude
df2['Longitude'] = Postcode_longitude

df2.head(15)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726245,-79.263670
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713133,-79.285055
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696665,-79.260163


In [14]:
df2.shape

(103, 5)